In [1]:
import pandas as pd
import sqlite3
import os
from IPython.display import display

# --- Step 1: 設定 ---
DB_PATH = "../data/processed/s2orc_filtered.db"
# 評価用50件バージョンの訓練データ論文リスト
TRAINING_DATAPAPERS_FILE = "../data/datapapers/sampled/training_data_papers_50.csv"
NEGATIVE_SAMPLE_RATIO = 4 # 正例1に対する負例の数

# --- Step 2: 訓練データセット規模の見積もり ---

def estimate_final_training_size_v2():
    """
    目視確認を各グループ1件行う前提で、構築可能な訓練データセットの規模を見積もる。
    """
    print("--- Estimating Final Training Set Size (v2: Based on 1 manual check per group) ---")

    if not os.path.exists(DB_PATH) or not os.path.exists(TRAINING_DATAPAPERS_FILE):
        print(f"❌ Error: Database or training data paper file not found.")
        return

    try:
        with sqlite3.connect(DB_PATH) as conn:
            # 1. 訓練用のデータ論文DOIリストを取得
            df_train_papers = pd.read_csv(TRAINING_DATAPAPERS_FILE)
            train_dois = tuple(df_train_papers['cited_datapaper_doi'].unique()) # SQL用にタプルに変換

            # 2. 各訓練用データ論文について、'Used'と判定された引用論文の数をカウント
            placeholders = ','.join('?' for _ in train_dois)
            query = f"""
                SELECT
                    cited_datapaper_doi,
                    COUNT(citing_doi) AS used_count
                FROM
                    positive_candidates
                WHERE
                    llm_annotation_status = 1
                    AND cited_datapaper_doi IN ({placeholders})
                GROUP BY
                    cited_datapaper_doi
                HAVING
                    COUNT(citing_doi) >= 2; -- ペア作成には最低2件必要
            """
            df_used_counts = pd.read_sql_query(query, conn, params=train_dois)

            if df_used_counts.empty:
                print("\nNo data papers in the training set can form positive pairs (need >= 2 'Used' papers).")
                return
            
            # 3. 作成可能なペア数を計算 (k件からk-1ペア)
            df_used_counts['creatable_pairs'] = df_used_counts['used_count'] - 1
            
            total_positive_pairs = df_used_counts['creatable_pairs'].sum()
            num_groups_to_check = len(df_used_counts) # 目視確認が必要なグループ数

            # --- 4. 結果の報告 ---
            print("\n" + "="*50)
            print("--- Estimation Results for Training Dataset ---")
            print(f"訓練データ作成の起点となるデータ論文数（元リスト）: {len(train_dois):,}")
            print(f"うち、正例ペアを作成可能なデータ論文の数 ('Used' >= 2): {num_groups_to_check:,}")
            print(f"⇒ 目視確認が必要な論文総数（最低）: {num_groups_to_check:,}件")
            print("-" * 50)
            print(f"作成可能な正例ペアの総数: {int(total_positive_pairs):,}")
            print("="*50)
            
            print(f"\n--- Total Dataset Size Estimation (Positive:Negative = 1:{NEGATIVE_SAMPLE_RATIO}) ---")
            total_negative_pairs = total_positive_pairs * NEGATIVE_SAMPLE_RATIO
            total_dataset_size = total_positive_pairs + total_negative_pairs
            print(f"  - Negative Pairs to be sampled: {int(total_negative_pairs):,}")
            print(f"  - Total Estimated Dataset Size: {int(total_dataset_size):,} pairs")

    except Exception as e:
        print(f"💥 An error occurred: {e}")

# --- 実行 ---
estimate_final_training_size_v2()

--- Estimating Final Training Set Size (v2: Based on 1 manual check per group) ---

--- Estimation Results for Training Dataset ---
訓練データ作成の起点となるデータ論文数（元リスト）: 150
うち、正例ペアを作成可能なデータ論文の数 ('Used' >= 2): 150
⇒ 目視確認が必要な論文総数（最低）: 150件
--------------------------------------------------
作成可能な正例ペアの総数: 1,438

--- Total Dataset Size Estimation (Positive:Negative = 1:4) ---
  - Negative Pairs to be sampled: 5,752
  - Total Estimated Dataset Size: 7,190 pairs


In [3]:
import pandas as pd
import sqlite3
import os
import math
from IPython.display import display

# --- Step 1: 設定 ---
DB_PATH = "../data/processed/s2orc_filtered.db"
TRAINING_DATAPAPERS_FILE = "../data/datapapers/sampled/training_data_papers_50.csv"

# ▼▼▼ 目標とする「追加」正例ペア数を設定 ▼▼▼
TARGET_ADDITIONAL_PAIRS = 1600

# ▼▼▼ 新規設定: 1つのデータ論文グループあたり、最大何件まで追加で確認するか ▼▼▼
MAX_CHECKS_PER_GROUP = 5 # 例えば、最大5件まで

# --- Step 2: 追加アノテーション計画のシミュレーション ---

def plan_balanced_additional_verification():
    """
    目標の追加ペア数に基づき、各グループの上限を考慮しながら、
    どのグループに追加で何件の目視確認を行うべきかを計画する。
    """
    print("--- Planning Balanced Additional Manual Verification ---")
    print(f"Target: Add approximately {TARGET_ADDITIONAL_PAIRS:,} more positive pairs.")
    print(f"Constraint: Max {MAX_CHECKS_PER_GROUP} additional checks per data paper group.")

    if not os.path.exists(DB_PATH) or not os.path.exists(TRAINING_DATAPAPERS_FILE):
        print(f"❌ Error: Database or training data paper file not found.")
        return

    try:
        # 1. 訓練用データ論文リストとその'Used'数を読み込む
        df_train = pd.read_csv(TRAINING_DATAPAPERS_FILE)
        if 'used_paper_count' not in df_train.columns:
             print("❌ Error: 'used_paper_count' column not found.")
             return
             
        # 2. 各グループで、追加確認した場合の「追加ペア数」を計算
        potential_gains = []
        for index, row in df_train.iterrows():
            k = row['used_paper_count']
            # 2件目から、最大で MAX_CHECKS_PER_GROUP + 1 件目までの確認を考慮
            # (すでに1件は確認済みなので、追加は最大 MAX_CHECKS_PER_GROUP 件)
            for m in range(2, min(k + 1, 2 + MAX_CHECKS_PER_GROUP)): # 修正点
                additional_pairs = k - m # m件目を確認した場合の追加ペア数
                if additional_pairs >= 0: # 修正点: >=0 に変更（最後の確認でもペアは0増える）
                    potential_gains.append({
                        'cited_datapaper_doi': row['cited_datapaper_doi'],
                        'verify_m_th_paper': m, # 何件目の確認か
                        'additional_pairs': additional_pairs,
                        'original_k': k
                    })
        
        if not potential_gains:
            print("No potential gains found.")
            return

        # 3. 追加ペア数が多い順にソート
        df_gains = pd.DataFrame(potential_gains).sort_values(by=['additional_pairs', 'original_k'], ascending=[False, False])

        # 4. 目標に達するまで、グループ上限を守りながらタスクを選択
        cumulative_pairs = 0
        tasks_to_perform = []
        checks_per_group = {} # 各グループで選択された追加確認数を記録

        for index, task in df_gains.iterrows():
            doi = task['cited_datapaper_doi']
            current_checks = checks_per_group.get(doi, 0)
            
            # グループごとの上限チェック
            if current_checks < MAX_CHECKS_PER_GROUP:
                tasks_to_perform.append(task)
                cumulative_pairs += task['additional_pairs']
                checks_per_group[doi] = current_checks + 1
                
                # 目標ペア数に達したら終了
                if cumulative_pairs >= TARGET_ADDITIONAL_PAIRS:
                    break
                
        # --- 5. 結果の報告 ---
        print("\n" + "="*50)
        print("--- Recommended Balanced Verification Plan ---")
        print(f"Total additional verifications needed: {len(tasks_to_perform):,} papers")
        print(f"Expected additional positive pairs: {int(cumulative_pairs):,} pairs")
        print("="*50)
        
        print("\n--- Details of Verification Tasks (Top 20 by gain) ---")
        display(pd.DataFrame(tasks_to_perform).head(20))
        
        print("\n--- Summary by Data Paper DOI (Sorted by additional checks) ---")
        df_summary = pd.DataFrame(tasks_to_perform)
        summary_table = df_summary.groupby('cited_datapaper_doi').agg(
            additional_verifications=('verify_m_th_paper', 'count'),
            max_m_verified=('verify_m_th_paper', 'max'),
            original_group_size=('original_k', 'first'),
            total_additional_pairs=('additional_pairs', 'sum')
        ).reset_index().sort_values(by='additional_verifications', ascending=False)
        
        display(summary_table.head(20)) # 表示件数を増やす

    except Exception as e:
        print(f"💥 An error occurred: {e}")

# --- 実行 ---
plan_balanced_additional_verification()

--- Planning Balanced Additional Manual Verification ---
Target: Add approximately 1,600 more positive pairs.
Constraint: Max 5 additional checks per data paper group.

--- Recommended Balanced Verification Plan ---
Total additional verifications needed: 16 papers
Expected additional positive pairs: 1,610 pairs

--- Details of Verification Tasks (Top 20 by gain) ---


,cited_datapaper_doi,verify_m_th_paper,additional_pairs,original_k
0,10.1038/S41597-020-0453-3,2,177,179
1,10.1038/S41597-020-0453-3,3,176,179
2,10.1038/S41597-020-0453-3,4,175,179
3,10.1038/S41597-020-0453-3,5,174,179
4,10.1038/S41597-020-0453-3,6,173,179
5,10.5194/ESSD-13-4349-2021,2,81,83
6,10.5194/ESSD-13-4349-2021,3,80,83
7,10.5194/ESSD-13-4349-2021,4,79,83
8,10.5194/ESSD-13-4349-2021,5,78,83
9,10.5194/ESSD-13-4349-2021,6,77,83



--- Summary by Data Paper DOI (Sorted by additional checks) ---


,cited_datapaper_doi,additional_verifications,max_m_verified,original_group_size,total_additional_pairs
1,10.1038/S41597-020-0453-3,5,6,179,875
2,10.1038/S41597-020-0534-3,5,6,62,290
3,10.5194/ESSD-13-4349-2021,5,6,83,395
0,10.1038/S41597-020-0369-Y,1,2,52,50
